In [1]:
import pandas as pd
import numpy as np
import funcs, gc, sqlite3

In [2]:
query = '''
SELECT    
    "RAT.NUM_ATIVIDADE",
    "MUNICIPIO",
    "LOGRADOURO",
    "DES_ENDERECO",
    "COMPLEMENTO_ENDERECO",
    "NOME_BAIRRO",
    "LOGRADOURO2",
    "DES_ENDERECO2",
    "SETOR"
FROM
    tbl_rat
WHERE
    "SETOR" == "other"
AND "MUNICIPIO" != "ITAUNA"
AND "NAT.CODIGO" IN ('Y07001', 'Y07003', 'Y07004', 'Y07005', 'Y04012')
'''

rats_unclassif = pd.read_sql(query, 'sqlite:///gdo.db')

In [3]:
df_classif = funcs.get_df_classif()
rats_unclassif.loc[:,'SETOR'] = rats_unclassif.apply(lambda row: funcs.classifica_setor(row, df_classif), axis=1)

In [4]:
new_classif_dict = {rats[0]:rats[-1] for rats in rats_unclassif.values}
new_classif_dict

{'2019-001822312-001': 'BOM PASTOR', '2019-010922203-001': 'NITEROI'}

In [17]:
query_update = '''
UPDATE tbl_rat
SET "SETOR" = ?
WHERE "RAT.NUM_ATIVIDADE" = ?
'''

with sqlite3.connect('gdo.db') as conn:
    cursor = conn.cursor()
    for nrat, setor in new_classif_dict.items():
        print((setor, nrat))
        cursor.execute(query_update, (setor, nrat))
        conn.commit()
conn.close()

In [18]:
rats = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')

In [19]:
rats[
    (rats['SETOR'] == 'other')
    & (rats['NAT.CODIGO'].str.contains('Y0700[1345]'))
    & (rats['MUNICIPIO'] != 'ITAUNA')
][[
    "RAT.NUM_ATIVIDADE",
    "MUNICIPIO",
    "LOGRADOURO",
    "DES_ENDERECO",
    "COMPLEMENTO_ENDERECO",
    "NOME_BAIRRO",
    "LOGRADOURO2",
    "DES_ENDERECO2",
    "SETOR"
]]

,RAT.NUM_ATIVIDADE,MUNICIPIO,LOGRADOURO,DES_ENDERECO,COMPLEMENTO_ENDERECO,NOME_BAIRRO,LOGRADOURO2,DES_ENDERECO2,SETOR
